In [ ]:
!pip install google-cloud-tasks==2.2.0

In [4]:
!gcloud config list

[core]
account = 798438782940-compute@developer.gserviceaccount.com
disable_usage_reporting = True
project = pascal-sandbox

Your active configuration is: [default]


In [5]:
import json
import csv
import argparse
import subprocess
import urllib.parse
from google.cloud import tasks_v2

In [6]:
# projet GCP
result = subprocess.run(['gcloud', 'config', 'get-value', 'project'], stdout=subprocess.PIPE)
project = result.stdout.decode('utf-8')[0:-1]
#print(project)
# liste des regions
result = subprocess.run(['cat', './regions-list.txt'], stdout=subprocess.PIPE)
regions = result.stdout.decode('utf-8').split('\n')
regions.pop()
#print(regions)
# liste des urls Cloud Run
result = subprocess.run(['cat', './urls-list.txt'], stdout=subprocess.PIPE)
urls = result.stdout.decode('utf-8').split('\n')
urls.pop()
for i in range(0,9):
    urls[i] = urls[i] + "/stroccurences"
    #print(urls)
    # list des tasks queues
    queues = []
    for r in regions:
        q = "wsreqsqueue-" + r
        queues.append(q)
        #print(queues)

In [7]:
# Create the clients
clients = []
for i in range(0,9):
    clients.append(tasks_v2.CloudTasksClient())
    # Construct the fully qualified queue names
parents = []
for i in range(0,9):
    parents.append(clients[i].queue_path(project, 'europe-west1', queues[i]))
print(parents)

['projects/pascal-sandbox/locations/europe-west1/queues/wsreqsqueue-asia-east1']
['projects/pascal-sandbox/locations/europe-west1/queues/wsreqsqueue-asia-east1', 'projects/pascal-sandbox/locations/europe-west1/queues/wsreqsqueue-asia-northeast1']
['projects/pascal-sandbox/locations/europe-west1/queues/wsreqsqueue-asia-east1', 'projects/pascal-sandbox/locations/europe-west1/queues/wsreqsqueue-asia-northeast1', 'projects/pascal-sandbox/locations/europe-west1/queues/wsreqsqueue-europe-north1']
['projects/pascal-sandbox/locations/europe-west1/queues/wsreqsqueue-asia-east1', 'projects/pascal-sandbox/locations/europe-west1/queues/wsreqsqueue-asia-northeast1', 'projects/pascal-sandbox/locations/europe-west1/queues/wsreqsqueue-europe-north1', 'projects/pascal-sandbox/locations/europe-west1/queues/wsreqsqueue-europe-west1']
['projects/pascal-sandbox/locations/europe-west1/queues/wsreqsqueue-asia-east1', 'projects/pascal-sandbox/locations/europe-west1/queues/wsreqsqueue-asia-northeast1', 'projec

In [8]:
def build_request(url, payload):
    task = {
        "view": tasks_v2.types.Task.View.FULL,
        "http_request": {  # Specify the type of request.
             #"http_method": tasks_v2.types.HttpMethod.POST,
             "url": url,  # The full url path that the task will be sent to.
             "oidc_token": {"service_account_email": "my-test-sa@pascal-sandbox.iam.gserviceaccount.com"},
         }
    }

    if payload is not None:
        if isinstance(payload, dict):
            # Convert dict to JSON string
            payload = json.dumps(payload)
            # specify http content-type to application/json
            task["http_request"]["headers"] = {"Content-type": "application/json"}

        # The API expects a payload of type bytes.
        converted_payload = payload.encode()

        # Add the payload to the request.
        task["http_request"]["body"] = converted_payload

    return task

In [35]:
def process_requests_file(tasks_client, parent, reqs_filename, run_url):
    print("Processing file: " + reqs_filename + " to queue: " + parent + " for run to service: " + run_url + "\n")
    with open(reqs_filename, newline='') as infile:
        rowreader = csv.reader(infile, delimiter=';')
        nb = 1
        for row in rowreader:
            if row[0]:
                if row[1]:
                    search = row[0] + " " + row[1]
                else:
                    search = row[0]
            else:
                search = row[1]
            #print(search)
            task = build_request(run_url, {'query': search, 'lexique': row[0], 'categorie': row[1]})
            #print("creating task for string " + search + " = lex: " + row[0] + " + cat: " + row[1])
            response = tasks_client.create_task(request={"parent": parent, "task": task})
            #print("Created task {}".format(response.name) +  "for string " + search)
            if nb % 1000 == 0:
                print(reqs_filename + " : " + str(nb))
                #print(search)
                #print(task)
            nb = nb + 1
    return "file {} processed".format(reqs_filename)

In [36]:
region = 0
process_requests_file(clients[region], parents[region], 'xaa', urls[region])

Processing file: xaa to queue: projects/pascal-sandbox/locations/europe-west1/queues/wsreqsqueue-asia-east1 for run to service: https://scrape-svc-asia-east1-7zookobxtq-de.a.run.app/stroccurences

xaa : 1000
xaa : 2000
xaa : 3000
xaa : 4000
xaa : 5000
xaa : 6000
xaa : 7000
xaa : 8000
xaa : 9000
xaa : 10000
xaa : 11000
xaa : 12000
xaa : 13000
xaa : 14000
xaa : 15000
xaa : 16000
xaa : 17000
xaa : 18000
xaa : 19000
xaa : 20000
xaa : 21000
xaa : 22000
xaa : 23000
xaa : 24000
xaa : 25000
xaa : 26000
xaa : 27000
xaa : 28000
xaa : 29000
xaa : 30000
xaa : 31000
xaa : 32000
xaa : 33000
xaa : 34000
xaa : 35000
xaa : 36000
xaa : 37000
xaa : 38000
xaa : 39000
xaa : 40000
xaa : 41000
xaa : 42000
xaa : 43000
xaa : 44000
xaa : 45000
xaa : 46000
xaa : 47000
xaa : 48000
xaa : 49000
xaa : 50000
xaa : 51000
xaa : 52000
xaa : 53000
xaa : 54000
xaa : 55000
xaa : 56000
xaa : 57000
xaa : 58000
xaa : 59000
xaa : 60000
xaa : 61000
xaa : 62000
xaa : 63000
xaa : 64000
xaa : 65000
xaa : 66000
xaa : 67000
xaa : 68

'file xaa processed'